In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

df=pd.read_csv("/content/OnlineRetail_cleaned.csv")

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [ ]:
#Convert InvoiceNo to string

df['InvoiceNo'] = df['InvoiceNo'].astype(str)

In [ ]:
#Prepare Basket Format
#Apriori needs a pivot table:
#Rows → Invoice
#Columns → Product Description
#Values → 0/1 (whether bought or not)

# Keep only needed columns
df = df[['InvoiceNo', 'Description', 'Quantity']]

basket = (df
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum()
          .unstack()
          .fillna(0))

# convert quantities to binary flags
basket = (basket > 0).astype(int)

basket.head()

Description,10 colour spaceboy pen,12 coloured party balloons,12 daisy pegs in wood box,12 egg house painted wood,12 hanging eggs hand painted,12 ivory rose peg place settings,12 message cards with envelopes,12 pencil small tube woodland,12 pencils small tube red retrospot,12 pencils small tube skull,12 pencils tall tube posy,12 pencils tall tube red retrospot,12 pencils tall tube skulls,12 pencils tall tube woodland,12 pink hen+chicks in basket,12 pink rose peg place settings,12 red rose peg place settings,15 pink fluffy chicks in box,15cm christmas glass ball 20 lights,16 pc cutlery set pantry design,16 piece cutlery set pantry design,18pc wooden cutlery set disposable,2 daisies hair comb,2 picture book eggs easter bunny,2 picture book eggs easter chicks,2 picture book eggs easter ducks,20 dolly pegs retrospot,200 bendy skull straws,200 red + white bendy straws,3 birds canvas screen,3 black cats w hearts blank card,3 drawer antique white wood cabinet,3 gardenia morris boxed candles,3 hearts hanging decoration rustic,3 hook hanger magic garden,3 hook photo shelf antique white,3 piece spaceboy cookie cutter set,3 pink hen+chicks in basket,3 raffia ribbons 50's christmas,3 raffia ribbons vintage christmas,...,yellow dragonfly helicopter,yellow easter egg hunt start post,yellow felt hanging heart w flower,yellow flowers felt handbag kit,yellow giant garden thermometer,yellow metal chicken heart,yellow pink flower design big bowl,yellow pot plant candle,yellow shark helicopter,yellow/blue retro radio,yellow/orange flower design plate,yellow/pink flower design big mug,you're confusing me metal sign,yuletide images gift wrap set,yuletide images s/6 paper boxes,zinc heart t-light holder,zinc star t-light holder,zinc box sign home,zinc finish 15cm planter pots,zinc folkart sleigh bells,zinc heart flower t-light holder,zinc heart lattice 2 wall planter,zinc heart lattice charger large,zinc heart lattice charger small,zinc heart lattice t-light holder,zinc heart lattice tray oval,zinc hearts plant pot holder,zinc herb garden container,zinc metal heart decoration,zinc plant pot holder,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
InvoiceNo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Use Apriori with a minimum support threshold.
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
frequent_itemsets.sort_values("support", ascending=False).head(10)

,support,itemsets
192,0.106357,(white hanging heart t-light holder)
154,0.091895,(regency cakestand 3 tier)
81,0.086337,(jumbo bag red retrospot)
124,0.074412,(party bunting)
11,0.074196,(assorted colour bird ornament)
102,0.069501,(lunch bag red retrospot)
166,0.061839,(set of 3 cake tins pantry design)
139,0.059303,(postage)
95,0.056767,(lunch bag black skull.)
116,0.055526,(pack of 72 retrospot cake cases)


In [ ]:


rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules = rules.sort_values("lift", ascending=False)


rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
74,(pink regency teacup and saucer),"(green regency teacup and saucer, roses regenc...",0.030002,0.029193,0.021045,0.701439,24.027846,1.0,0.020169,3.251619,0.988024,0.551627,0.692461,0.711163
71,"(green regency teacup and saucer, roses regenc...",(pink regency teacup and saucer),0.029193,0.030002,0.021045,0.720887,24.027846,1.0,0.020169,3.475290,0.987201,0.551627,0.712254,0.711163
75,(green regency teacup and saucer),"(pink regency teacup and saucer, roses regency...",0.037287,0.023527,0.021045,0.564399,23.989564,1.0,0.020167,2.241671,0.995432,0.529172,0.553904,0.729447
70,"(pink regency teacup and saucer, roses regency...",(green regency teacup and saucer),0.023527,0.037287,0.021045,0.894495,23.989564,1.0,0.020167,9.124846,0.981405,0.529172,0.890409,0.729447
8,(pink regency teacup and saucer),(green regency teacup and saucer),0.030002,0.037287,0.024822,0.827338,22.188466,1.0,0.023703,5.575714,0.984468,0.584498,0.820651,0.746520
9,(green regency teacup and saucer),(pink regency teacup and saucer),0.037287,0.030002,0.024822,0.665702,22.188466,1.0,0.023703,2.901595,0.991917,0.584498,0.655362,0.746520
73,(roses regency teacup and saucer),"(pink regency teacup and saucer, green regency...",0.042251,0.024822,0.021045,0.498084,20.066300,1.0,0.019996,1.942912,0.992082,0.457210,0.485309,0.672955
72,"(pink regency teacup and saucer, green regency...",(roses regency teacup and saucer),0.024822,0.042251,0.021045,0.847826,20.066300,1.0,0.019996,6.293778,0.974350,0.457210,0.841113,0.672955
62,(pink regency teacup and saucer),(roses regency teacup and saucer),0.030002,0.042251,0.023527,0.784173,18.559754,1.0,0.022259,4.437569,0.975384,0.482835,0.774651,0.670503
63,(roses regency teacup and saucer),(pink regency teacup and saucer),0.042251,0.030002,0.023527,0.556833,18.559754,1.0,0.022259,2.188785,0.987858,0.482835,0.543125,0.670503


In [ ]:
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(15)


,antecedents,consequents,support,confidence,lift
74,(pink regency teacup and saucer),"(green regency teacup and saucer, roses regenc...",0.021045,0.701439,24.027846
71,"(green regency teacup and saucer, roses regenc...",(pink regency teacup and saucer),0.021045,0.720887,24.027846
75,(green regency teacup and saucer),"(pink regency teacup and saucer, roses regency...",0.021045,0.564399,23.989564
70,"(pink regency teacup and saucer, roses regency...",(green regency teacup and saucer),0.021045,0.894495,23.989564
8,(pink regency teacup and saucer),(green regency teacup and saucer),0.024822,0.827338,22.188466
9,(green regency teacup and saucer),(pink regency teacup and saucer),0.024822,0.665702,22.188466
73,(roses regency teacup and saucer),"(pink regency teacup and saucer, green regency...",0.021045,0.498084,20.066300
72,"(pink regency teacup and saucer, green regency...",(roses regency teacup and saucer),0.021045,0.847826,20.066300
62,(pink regency teacup and saucer),(roses regency teacup and saucer),0.023527,0.784173,18.559754
63,(roses regency teacup and saucer),(pink regency teacup and saucer),0.023527,0.556833,18.559754


In [ ]:
#Apriori outputs sets. Converting them for clean presentation.

rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

rules.head()


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
74,pink regency teacup and saucer,"green regency teacup and saucer, roses regency...",0.030002,0.029193,0.021045,0.701439,24.027846,1.0,0.020169,3.251619,0.988024,0.551627,0.692461,0.711163
71,"green regency teacup and saucer, roses regency...",pink regency teacup and saucer,0.029193,0.030002,0.021045,0.720887,24.027846,1.0,0.020169,3.475290,0.987201,0.551627,0.712254,0.711163
75,green regency teacup and saucer,"pink regency teacup and saucer, roses regency ...",0.037287,0.023527,0.021045,0.564399,23.989564,1.0,0.020167,2.241671,0.995432,0.529172,0.553904,0.729447
70,"pink regency teacup and saucer, roses regency ...",green regency teacup and saucer,0.023527,0.037287,0.021045,0.894495,23.989564,1.0,0.020167,9.124846,0.981405,0.529172,0.890409,0.729447
8,pink regency teacup and saucer,green regency teacup and saucer,0.030002,0.037287,0.024822,0.827338,22.188466,1.0,0.023703,5.575714,0.984468,0.584498,0.820651,0.746520


In [ ]:
rules.to_csv("Association_Rules_Online_Retail.csv", index=False)
